# PARTE 1 - Pré-Processamento
Esse parte **Jupyter Notebook** tem como objetivo aplicar um **Pré-Processamento** no conjunto de dados.

# Introdução ao Pré-Processamento Iterativo
Durante o *Pré-Processamento* utilizarei uma approach que eu chamo de ***Pré-Processamento iterativo***. Isso, porque eu vou trabalhar cada *coluna (feature)* do conjunto de dados individualmente e vou iterando por cada uma aplicando várias técnicas (quando necessário).

Algo parecido com isso:

![img](../images/loop.png)

**NOTE:**  
A imagem não ficou muito bonita, mas o importante é vocês entenderem como vai ser o processo que vamos seguir.

# Resumo da Análise preliminar
Na etapa anterior foi feita uma breve análise do conjunto de dados. O **resumo** dessa análise foi o seguinte:

 - Temos um grande conjunto de dados para trabalharmos:
   - Com 244.768 amostras e 12 colunas (features).
 - Porém, vai ser necessário um Pré-Processamento na maior parte das colunas, devido o fato das colunas serem representadas por textos (informações).
 - Algumas colunas estão com muitos dados faltantes, principalmente a **ContractType** que tem **73%** dos dados faltantes.

# 01 - Imports & Configurações
Vamos começar com importações e configurações iniciais:

## Importando as bibliotecas necessárias:

In [79]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import wordnet
from scipy.sparse import hstack
import scipy.sparse
import pandas as pd
import numpy as np
import string
import spacy
import py7zr
import nltk
import re

## Extraindo o conjunto de dados:

In [2]:
with py7zr.SevenZipFile("../datasets/Train_rev1.7z", mode='r') as archive:
  archive.extractall(path="/tmp") # For Linux users.

## Configurando o tamanho das saídas (outputs):
Antes de iniciarmos nossa análise vamos configurar o Pandas para exibir todo o conteúdo por amostra:

In [3]:
pd.options.display.max_colwidth = 100000

## Pegando o conjunto de dados

In [4]:
full_df = pd.read_csv("/tmp/Train_rev1.csv")

# 02 - Visão geral (overview) dos conjunto de dados
Vamos começar com uma visão geral (overview) dos dados, diferente da approach que vamos seguir **(Pré-Processamento Iterativo)** que vai tratar cada coluna (feature) individualmente aqui vamos dar uma breve visualizada no conjunto de dados como um todo e sem nenhum Processamento:

In [5]:
full_df.info()
full_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  244768 non-null  int64 
 1   Title               244767 non-null  object
 2   FullDescription     244768 non-null  object
 3   LocationRaw         244768 non-null  object
 4   LocationNormalized  244768 non-null  object
 5   ContractType        65442 non-null   object
 6   ContractTime        180863 non-null  object
 7   Company             212338 non-null  object
 8   Category            244768 non-null  object
 9   SalaryRaw           244768 non-null  object
 10  SalaryNormalized    244768 non-null  int64 
 11  SourceName          244767 non-null  object
dtypes: int64(2), object(10)
memory usage: 22.4+ MB


,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K","Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****","Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,"Mathematical Modeller / Simulation Analyst / Operational Analyst Basingstoke, Hampshire Up to ****K AAE pension contribution, private medical and dental The opportunity Our client is an independent consultancy firm which has an opportunity for a Data Analyst with 35 years experience. The role will require the successful candidate to demonstrate their ability to analyse a problem and arrive at a solution, with varying levels of data being available. Essential skills Thorough knowledge of Excel and proven ability to utilise this to create powerful decision support models Experience in Modelling and Simulation Techniques, Experience of techniques such as Discrete Event Simulation and/or SD modelling Mathematical/scientific background minimum degree qualified Proven analytical and problem solving skills Self Starter Ability to develop solid working relationships In addition to formal qualifications and experience, the successful candidate will require excellent written and verbal communication skills, be energetic, enterprising and have a determination to succeed. They will be required to build solid working relationships, both internally with colleagues and, most importantly, externally with our clients. They must be comfortable working independently to deliver against challenging client demands. The offices are located in Basingstoke, Hampshire, but our client work for clients worldwide. The successful candidate must therefore

# 03 - Pré-Processamento Iterativo
Agora sim, nós vamos *entender* e *processar* cada colunas (features) individualmente seguindo nossa approach *iterativa*.

**Essa abordagem segue 2 linhas de pensamento:**
 - **Dividir para conquistar:**
   - Onde, nós vamos dividir o problema em problema menores (colunas/features);
   - Depois de resolvido cada parte individualmente, juntamos todos os pedaços e temos uma solução final.
 - **Aplicar "x" técnica por iteração:**
   - Por exemplo na **primeira iteração** nós vamos adequar os tipos de dados para cada coluna (feature);
   - Na **segunda iteração** nós vamos ver quanto porcento de dados faltantes tem por coluna (feature) e aplicar alguma técnicas que resolva esse problema (quando necessário);
   - ...
   - Essas iterações vão seguindo até o fim do Pré-Processamento.

## 03.1 - Pré-Processando a coluna (feature) "Id"
> Essa coluna (feature) não vai precisar ser Pré-Processada. Como nós sabemos é apenas o identificado único de cada amostra.

In [6]:
full_df['Id'].head()

0    12612628
1    12612830
2    12612844
3    12613049
4    12613647
Name: Id, dtype: int64

---

## 03.2 - Pré-Processando a coluna (feature) "Title"
> Resumidamente, o **Title** é o resumo do *cargo* ou *função*.

### Preparando e colocando o tipo de dado mais adequado na *coluna (feature)* "title":

In [7]:
df_title = full_df[["Title"]]
df_title = df_title.astype({'Title': 'string'})
df_title.info()
df_title.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Title   244767 non-null  string
dtypes: string(1)
memory usage: 1.9 MB


,Title
0,Engineering Systems Analyst
1,Stress Engineer Glasgow
2,Modelling and simulation analyst
3,Engineering Systems Analyst / Mathematical Modeller
4,"Pioneer, Miser Engineering Systems Analyst"


### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [8]:
# Data missing sum.
missing = df_title.isnull().sum()
missing

Title    1
dtype: int64

In [9]:
# Data missing in percent.
percentMissing = (missing / len(df_title.index)) * 100
percentMissing

Title    0.000409
dtype: float64

### Aplicando Lower Casing:

In [10]:
df_title["processed_title"] = df_title["Title"].str.lower()
df_title.head()

,Title,processed_title
0,Engineering Systems Analyst,engineering systems analyst
1,Stress Engineer Glasgow,stress engineer glasgow
2,Modelling and simulation analyst,modelling and simulation analyst
3,Engineering Systems Analyst / Mathematical Modeller,engineering systems analyst / mathematical modeller
4,"Pioneer, Miser Engineering Systems Analyst","pioneer, miser engineering systems analyst"


### Removendo pontuações:

In [11]:
df_title["processed_title"] = df_title["processed_title"].str.replace('[^\w\s]',' ', regex=True)
df_title.head()

,Title,processed_title
0,Engineering Systems Analyst,engineering systems analyst
1,Stress Engineer Glasgow,stress engineer glasgow
2,Modelling and simulation analyst,modelling and simulation analyst
3,Engineering Systems Analyst / Mathematical Modeller,engineering systems analyst mathematical modeller
4,"Pioneer, Miser Engineering Systems Analyst",pioneer miser engineering systems analyst


### Removendo números:

In [12]:
df_title["processed_title"] = df_title["processed_title"].str.replace('[0-9]+', '', regex=True)
df_title.head()

,Title,processed_title
0,Engineering Systems Analyst,engineering systems analyst
1,Stress Engineer Glasgow,stress engineer glasgow
2,Modelling and simulation analyst,modelling and simulation analyst
3,Engineering Systems Analyst / Mathematical Modeller,engineering systems analyst mathematical modeller
4,"Pioneer, Miser Engineering Systems Analyst",pioneer miser engineering systems analyst


### Aplicando Stopword Removal (Remoção de palavras irrelevantes)

In [13]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [14]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
  return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df_title["processed_title"] = df_title["processed_title"].apply(lambda text: remove_stopwords(text))
df_title.head()

,Title,processed_title
0,Engineering Systems Analyst,engineering systems analyst
1,Stress Engineer Glasgow,stress engineer glasgow
2,Modelling and simulation analyst,modelling simulation analyst
3,Engineering Systems Analyst / Mathematical Modeller,engineering systems analyst mathematical modeller
4,"Pioneer, Miser Engineering Systems Analyst",pioneer miser engineering systems analyst


### Removendo palavras mais frequentes

In [15]:
cnt_title = Counter() # Instance
for text in df_title["processed_title"].values:
  for word in text.split():
    cnt_title[word] += 1

cnt_title.most_common(10)

[('manager', 50162),
 ('engineer', 24192),
 ('sales', 19769),
 ('senior', 16976),
 ('developer', 13895),
 ('assistant', 12179),
 ('k', 11057),
 ('executive', 10632),
 ('business', 9988),
 ('consultant', 9496)]

**NOTE:**  
Na minha opinião quase todas, senão todas (tirandi "k") são relevantes para o modelo aprender. Sabendo disso não vou remover nenhuma delas.

### Remoção de palavras raras:

In [16]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt_title.most_common()[:-n_rare_words-1:-1]])
RAREWORDS

{'bellhill',
 'constructions',
 'hydrolic',
 'improvemen',
 'leadopportunity',
 'mlnlycke',
 'norley',
 'tase',
 'techniciancivil',
 'tuiton'}

In [17]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt_title.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
  return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df_title["processed_title"] = df_title["processed_title"].apply(lambda text: remove_rarewords(text))
df_title.head()

,Title,processed_title
0,Engineering Systems Analyst,engineering systems analyst
1,Stress Engineer Glasgow,stress engineer glasgow
2,Modelling and simulation analyst,modelling simulation analyst
3,Engineering Systems Analyst / Mathematical Modeller,engineering systems analyst mathematical modeller
4,"Pioneer, Miser Engineering Systems Analyst",pioneer miser engineering systems analyst


### Aplicando a técnica de Stemming:

In [18]:
stemmer = PorterStemmer() # Instance.
def stem_words(text):
  return " ".join([stemmer.stem(word) for word in text.split()])

df_title["processed_title"] = df_title["processed_title"].apply(lambda text: stem_words(text))
df_title.head()

,Title,processed_title
0,Engineering Systems Analyst,engin system analyst
1,Stress Engineer Glasgow,stress engin glasgow
2,Modelling and simulation analyst,model simul analyst
3,Engineering Systems Analyst / Mathematical Modeller,engin system analyst mathemat model
4,"Pioneer, Miser Engineering Systems Analyst",pioneer miser engin system analyst


### Aplicando a técnica de Lemmatization + Part-of-Speech Tagging:

In [19]:
lemmatizer = WordNetLemmatizer() # Instance
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV} # Apply dict mapping.

# Lemmatize words function.
def lemmatize_words(text):
  pos_tagged_text = nltk.pos_tag(text.split())
  return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df_title["processed_title"] = df_title["processed_title"].apply(lambda text: lemmatize_words(text))
df_title.head()

,Title,processed_title
0,Engineering Systems Analyst,engin system analyst
1,Stress Engineer Glasgow,stress engin glasgow
2,Modelling and simulation analyst,model simul analyst
3,Engineering Systems Analyst / Mathematical Modeller,engin system analyst mathemat model
4,"Pioneer, Miser Engineering Systems Analyst",pioneer miser engin system analyst


### Aplicando a técnica de Count Vectorizer

In [24]:
vectorizer = CountVectorizer() # Instance.
df_title_vectorized = vectorizer.fit_transform(df_title["processed_title"])

### Salvando a Matriz esparsa

In [26]:
scipy.sparse.save_npz('df_title_vectorized.npz', df_title_vectorized)

---

## 03.3 - Pré-Processando a coluna (feature) "FullDescription"
>O texto completo do anúncio de emprego, conforme fornecido pelo anunciante do emprego.

**NOTE:**  
Onde teria o salário (salary) qual foi retirado os valores da descrição para garantir que nenhuma informação de salário apareça nas descrições. Pode haver algum dano colateral aqui, pois também foi removido outros números.

### Preparando e colocando o tipo de dado mais adequado na *coluna (feature)* "FullDescription":

In [27]:
df_FullDescription = full_df[["FullDescription"]]
df_FullDescription = df_FullDescription.astype({'FullDescription': 'string'})
df_FullDescription.info()
df_FullDescription.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   FullDescription  244768 non-null  string
dtypes: string(1)
memory usage: 1.9 MB


,FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K"
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****"
2,"Mathematical Modeller / Simulation Analyst / Operational Analyst Basingstoke, Hampshire Up to ****K AAE pension contribution, private medical and dental The opportunity Our client is an independent consultancy firm which has an opportunity for a Data Analyst with 35 years experience. The role will require the successful candidate to demonstrate their ability to analyse a problem and arrive at a solution, with varying levels of data being available. Essential skills Thorough knowledge of Excel and proven ability to utilise this to create powerful decision support models Experience in Modelling and Simulation Techniques, Experience of techniques such as Discrete Event Simulation and/or SD modelling Mathematical/scientific background minimum degree qualified Proven analytical and problem solving skills Self Starter Ability to develop solid working relationships In addition to formal qualifications and experience, the successful candidate will require excellent written and verbal communication skills, be energetic, enterprising and have a determination to succeed. They will be required to build solid working relationships, both internally with colleagues and, most importantly, externally with our clients. They must be comfortable working independently to deliver against challenging client demands. The offices are located in Basingstoke, Hampshire, but our client work for clients worldwide. The successful candidate must therefore be prepared to undertake work at client sites for short periods of time. Physics, Mathematics, Modelling, Simulation, Analytical, Operational Research, Mathematical Modelling Mathematical Modeller / Simulation Analyst / Operational Analyst Basingstoke, Hampshire ****K AAE pension contribution, private medical and dental"
3,"Engineering Systems Analyst / Mathematical Modeller. Our client is a highly successful and respected Consultancy providing specialist software development MISER, PIONEER, Maths, Mathematical, Optimi

### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [28]:
# Data missing sum.
missing = df_FullDescription.isnull().sum()
missing

FullDescription    0
dtype: int64

In [29]:
# Data missing in percent.
percentMissing = (missing / len(df_FullDescription.index)) * 100
percentMissing

FullDescription    0.0
dtype: float64

### Aplicando Lower Casing:

In [30]:
df_FullDescription["processed_FullDescription"] = df_FullDescription["FullDescription"].str.lower()
df_FullDescription.head()

,FullDescription,processed_FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K","engineering systems analyst dorking surrey salary ****k our client is located in dorking, surrey and are looking for engineering systems analyst our client provides specialist software development keywords mathematical modelling, risk analysis, system modelling, optimisation, miser, pioneeer engineering systems analyst dorking surrey salary ****k"
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****","stress engineer glasgow salary **** to **** we re currently looking for talented engineers to join our growing glasgow team at a variety of levels. the roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. in return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain chartership and some opportunities to possibly travel or work in other offices, in or outside of the uk. the requirements you will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. you will need to demonstrate experience in at least one or more of the following areas: structural/stress analysis composite stress analysis (any industry) linear and nonlinear finite element analysis fatigue and damage tolerance structural dynamics thermal analysis aerostructures experience you will also be expected to demonstrate the following qualities: a strong desire to progress quickly to a position of leadership professional approach strong communication skills, written and verbal commercial awareness team working, being comfortable working in international teams and self managing please note security clearance is required for th

### Removendo URLs:

In [31]:
df_FullDescription["processed_FullDescription"] = df_FullDescription["processed_FullDescription"].str.replace(r'\s*https?://\S+(\s+|$)', ' ', regex=True).str.strip()
df_FullDescription.head()

,FullDescription,processed_FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K","engineering systems analyst dorking surrey salary ****k our client is located in dorking, surrey and are looking for engineering systems analyst our client provides specialist software development keywords mathematical modelling, risk analysis, system modelling, optimisation, miser, pioneeer engineering systems analyst dorking surrey salary ****k"
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****","stress engineer glasgow salary **** to **** we re currently looking for talented engineers to join our growing glasgow team at a variety of levels. the roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. in return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain chartership and some opportunities to possibly travel or work in other offices, in or outside of the uk. the requirements you will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. you will need to demonstrate experience in at least one or more of the following areas: structural/stress analysis composite stress analysis (any industry) linear and nonlinear finite element analysis fatigue and damage tolerance structural dynamics thermal analysis aerostructures experience you will also be expected to demonstrate the following qualities: a strong desire to progress quickly to a position of leadership professional approach strong communication skills, written and verbal commercial awareness team working, being comfortable working in international teams and self managing please note security clearance is required for th

### Removendo pontuações:

In [32]:
df_FullDescription["processed_FullDescription"] = df_FullDescription["processed_FullDescription"].str.replace('[^\w\s]',' ', regex=True)
df_FullDescription.head(50)

,FullDescription,processed_FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K",engineering systems analyst dorking surrey salary k our client is located in dorking surrey and are looking for engineering systems analyst our client provides specialist software development keywords mathematical modelling risk analysis system modelling optimisation miser pioneeer engineering systems analyst dorking surrey salary k
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****",stress engineer glasgow salary to we re currently looking for talented engineers to join our growing glasgow team at a variety of levels the roles are ideally suited to high calibre engineering graduates with any level of appropriate experience so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects spanning both aerostructures and aeroengines in return you can expect good career opportunities and the chance for advancement and personal and professional development support while you gain chartership and some opportunities to possibly travel or work in other offices in or outside of the uk the requirements you will need to have a good engineering degree that includes structural analysis such as aeronautical mechanical automotive civil with some experience in a professional engineering environment relevant to but not limited to the aerospace sector you will need to demonstrate experience in at least one or more of the following areas structural stress analysis composite stress analysis any industry linear and nonlinear finite element analysis fatigue and damage tolerance structural dynamics thermal analysis aerostructures experience you will also be expected to demonstrate the following qualities a strong desire to progress quickly to a position of leadership professional approach strong communication skills written and verbal commercial awareness team working being comfortable working in international teams and self managing please note security clearance is required for this role stress engineer glasgow salary to
2,"Math

### Removendo números:

In [33]:
df_FullDescription["processed_FullDescription"] = df_FullDescription["processed_FullDescription"].str.replace('[0-9]+', '', regex=True)
df_FullDescription.head()

,FullDescription,processed_FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K",engineering systems analyst dorking surrey salary k our client is located in dorking surrey and are looking for engineering systems analyst our client provides specialist software development keywords mathematical modelling risk analysis system modelling optimisation miser pioneeer engineering systems analyst dorking surrey salary k
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****",stress engineer glasgow salary to we re currently looking for talented engineers to join our growing glasgow team at a variety of levels the roles are ideally suited to high calibre engineering graduates with any level of appropriate experience so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects spanning both aerostructures and aeroengines in return you can expect good career opportunities and the chance for advancement and personal and professional development support while you gain chartership and some opportunities to possibly travel or work in other offices in or outside of the uk the requirements you will need to have a good engineering degree that includes structural analysis such as aeronautical mechanical automotive civil with some experience in a professional engineering environment relevant to but not limited to the aerospace sector you will need to demonstrate experience in at least one or more of the following areas structural stress analysis composite stress analysis any industry linear and nonlinear finite element analysis fatigue and damage tolerance structural dynamics thermal analysis aerostructures experience you will also be expected to demonstrate the following qualities a strong desire to progress quickly to a position of leadership professional approach strong communication skills written and verbal commercial awareness team working being comfortable working in international teams and self managing please note security clearance is required for this role stress engineer glasgow salary to
2,"Math

### Aplicacando Stopword Removal (Remoção de palavras irrelevantes)

In [34]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [35]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
  return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df_FullDescription["processed_FullDescription"] = df_FullDescription["processed_FullDescription"].apply(lambda text: remove_stopwords(text))
df_FullDescription.head()

,FullDescription,processed_FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K",engineering systems analyst dorking surrey salary k client located dorking surrey looking engineering systems analyst client provides specialist software development keywords mathematical modelling risk analysis system modelling optimisation miser pioneeer engineering systems analyst dorking surrey salary k
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****",stress engineer glasgow salary currently looking talented engineers join growing glasgow team variety levels roles ideally suited high calibre engineering graduates level appropriate experience give opportunity use technical skills provide high quality input aerospace projects spanning aerostructures aeroengines return expect good career opportunities chance advancement personal professional development support gain chartership opportunities possibly travel work offices outside uk requirements need good engineering degree includes structural analysis aeronautical mechanical automotive civil experience professional engineering environment relevant limited aerospace sector need demonstrate experience least one following areas structural stress analysis composite stress analysis industry linear nonlinear finite element analysis fatigue damage tolerance structural dynamics thermal analysis aerostructures experience also expected demonstrate following qualities strong desire progress quickly position leadership professional approach strong communication skills written verbal commercial awareness team working comfortable working international teams self managing please note security clearance required role stress engineer glasgow salary
2,"Mathematical Modeller / Simulation Analyst / Operational Analyst Basingstoke, Hampshire Up to ****K AAE pension contribution, private medical and dental The opportunity Our client is an independent consultancy firm which has an opportunity for a Data Analyst with 35 years experience. The role will require the successful candidate to demonstrate their ability to analyse a problem and a

### Removendo palavras mais frequentes

In [36]:
cnt_fulldescription = Counter() # Instance
for text in df_FullDescription["processed_FullDescription"].values:
  for word in text.split():
    cnt_fulldescription[word] += 1

cnt_fulldescription.most_common(10)

[('experience', 428061),
 ('role', 292124),
 ('work', 279781),
 ('team', 268361),
 ('business', 265990),
 ('skills', 235950),
 ('working', 222664),
 ('within', 217339),
 ('sales', 209318),
 ('client', 197545)]

**NOTE:**  
Na minha opinião quase todas, senão todas são relevantes para o modelo aprender. Sabendo disso não vou remover nenhuma delas.

### Remoção de palavras raras:

In [37]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt_fulldescription.most_common()[:-n_rare_words-1:-1]])
RAREWORDS

{'gpled',
 'grzedamcarthur',
 'grzedamcarthurnhs',
 'immedaitley',
 'lowehays',
 'organsations',
 'ruabon',
 'stephanietraveltraderecruitmnt',
 'swuk',
 'tne'}

In [38]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt_fulldescription.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
  return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df_FullDescription["processed_FullDescription"] = df_FullDescription["processed_FullDescription"].apply(lambda text: remove_rarewords(text))
df_FullDescription.head()

,FullDescription,processed_FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K",engineering systems analyst dorking surrey salary k client located dorking surrey looking engineering systems analyst client provides specialist software development keywords mathematical modelling risk analysis system modelling optimisation miser pioneeer engineering systems analyst dorking surrey salary k
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****",stress engineer glasgow salary currently looking talented engineers join growing glasgow team variety levels roles ideally suited high calibre engineering graduates level appropriate experience give opportunity use technical skills provide high quality input aerospace projects spanning aerostructures aeroengines return expect good career opportunities chance advancement personal professional development support gain chartership opportunities possibly travel work offices outside uk requirements need good engineering degree includes structural analysis aeronautical mechanical automotive civil experience professional engineering environment relevant limited aerospace sector need demonstrate experience least one following areas structural stress analysis composite stress analysis industry linear nonlinear finite element analysis fatigue damage tolerance structural dynamics thermal analysis aerostructures experience also expected demonstrate following qualities strong desire progress quickly position leadership professional approach strong communication skills written verbal commercial awareness team working comfortable working international teams self managing please note security clearance required role stress engineer glasgow salary
2,"Mathematical Modeller / Simulation Analyst / Operational Analyst Basingstoke, Hampshire Up to ****K AAE pension contribution, private medical and dental The opportunity Our client is an independent consultancy firm which has an opportunity for a Data Analyst with 35 years experience. The role will require the successful candidate to demonstrate their ability to analyse a problem and a

### Aplica a técnica de Stemming

In [39]:
stemmer = PorterStemmer() # Instance.
def stem_words(text):
  return " ".join([stemmer.stem(word) for word in text.split()])

df_FullDescription["processed_FullDescription"] = df_FullDescription["processed_FullDescription"].apply(lambda text: stem_words(text))
df_FullDescription.head()

,FullDescription,processed_FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K",engin system analyst dork surrey salari k client locat dork surrey look engin system analyst client provid specialist softwar develop keyword mathemat model risk analysi system model optimis miser pioneeer engin system analyst dork surrey salari k
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****",stress engin glasgow salari current look talent engin join grow glasgow team varieti level role ideal suit high calibr engin graduat level appropri experi give opportun use technic skill provid high qualiti input aerospac project span aerostructur aeroengin return expect good career opportun chanc advanc person profession develop support gain chartership opportun possibl travel work offic outsid uk requir need good engin degre includ structur analysi aeronaut mechan automot civil experi profession engin environ relev limit aerospac sector need demonstr experi least one follow area structur stress analysi composit stress analysi industri linear nonlinear finit element analysi fatigu damag toler structur dynam thermal analysi aerostructur experi also expect demonstr follow qualiti strong desir progress quickli posit leadership profession approach strong commun skill written verbal commerci awar team work comfort work intern team self manag pleas note secur clearanc requir role stress engin glasgow salari
2,"Mathematical Modeller / Simulation Analyst / Operational Analyst Basingstoke, Hampshire Up to ****K AAE pension contribution, private medical and dental The opportunity Our client is an independent consultancy firm which has an opportunity for a Data Analyst with 35 years experience. The role will require the successful candidate to demonstrate their ability to analyse a problem and arrive at a solution, with varying levels of data being available. Essential skills Thorough knowledge of Excel and proven ability to utilise this to create powerful decision support models Experience in Modelling and Simulation Techniques, Experience of techniques such as Discrete Event Simula

### Aplica a técnica de Lemmatization

In [40]:
lemmatizer = WordNetLemmatizer() # Instance.
def lemmatize_words(text):
  return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df_FullDescription["processed_FullDescription"] = df_FullDescription["processed_FullDescription"].apply(lambda text: lemmatize_words(text))
df_FullDescription.head()

,FullDescription,processed_FullDescription
0,"Engineering Systems Analyst Dorking Surrey Salary ****K Our client is located in Dorking, Surrey and are looking for Engineering Systems Analyst our client provides specialist software development Keywords Mathematical Modelling, Risk Analysis, System Modelling, Optimisation, MISER, PIONEEER Engineering Systems Analyst Dorking Surrey Salary ****K",engin system analyst dork surrey salari k client locat dork surrey look engin system analyst client provid specialist softwar develop keyword mathemat model risk analysi system model optimis miser pioneeer engin system analyst dork surrey salari k
1,"Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to demonstrate experience in at least one or more of the following areas: Structural/stress analysis Composite stress analysis (any industry) Linear and nonlinear finite element analysis Fatigue and damage tolerance Structural dynamics Thermal analysis Aerostructures experience You will also be expected to demonstrate the following qualities: A strong desire to progress quickly to a position of leadership Professional approach Strong communication skills, written and verbal Commercial awareness Team working, being comfortable working in international teams and self managing PLEASE NOTE SECURITY CLEARANCE IS REQUIRED FOR THIS ROLE Stress Engineer Glasgow Salary **** to ****",stress engin glasgow salari current look talent engin join grow glasgow team varieti level role ideal suit high calibr engin graduat level appropri experi give opportun use technic skill provid high qualiti input aerospac project span aerostructur aeroengin return expect good career opportun chanc advanc person profession develop support gain chartership opportun possibl travel work offic outsid uk requir need good engin degre includ structur analysi aeronaut mechan automot civil experi profession engin environ relev limit aerospac sector need demonstr experi least one follow area structur stress analysi composit stress analysi industri linear nonlinear finit element analysi fatigu damag toler structur dynam thermal analysi aerostructur experi also expect demonstr follow qualiti strong desir progress quickli posit leadership profession approach strong commun skill written verbal commerci awar team work comfort work intern team self manag plea note secur clearanc requir role stress engin glasgow salari
2,"Mathematical Modeller / Simulation Analyst / Operational Analyst Basingstoke, Hampshire Up to ****K AAE pension contribution, private medical and dental The opportunity Our client is an independent consultancy firm which has an opportunity for a Data Analyst with 35 years experience. The role will require the successful candidate to demonstrate their ability to analyse a problem and arrive at a solution, with varying levels of data being available. Essential skills Thorough knowledge of Excel and proven ability to utilise this to create powerful decision support models Experience in Modelling and Simulation Techniques, Experience of techniques such as Discrete Event Simulat

### Aplica a técnica de Count Vectorizer

In [41]:
vectorizer = CountVectorizer() # Instance.
df_FullDescription_vectorized = vectorizer.fit_transform(df_FullDescription["processed_FullDescription"])

### Salvando a Matriz esparsa

In [42]:
scipy.sparse.save_npz('df_fulldescription_vectorized.npz', df_FullDescription_vectorized)

---

## 03.4 - Pré-Processando a coluna (feature) "LocationRaw"
> Imagine que essa coluna representa a localização da vaga, porém, utilizando pontos cardeais e/ou referências.

**NOTE:**  
Como eu já tenho uma coluna (feature) igual a essa, porém, normalizada eu vou ignorar essa.

---

## 03.5 - Pré-Processando a coluna (feature) "LocationNormalized"
> Tem o mesmo significado da coluna **LocationRaw**, porém, com menos informações e referências.

**NOTE:**  
Isso, porque essa coluna é o resultado de um Pré-Processamento da coluna **LocationRaw** feito pelo **Adzuna**.

### Preparando e colocando o tipo de dado mais adequado a *coluna (feature)* "LocationNormalized":

In [43]:
df_LocationNormalized = full_df[["LocationNormalized"]]
df_LocationNormalized = df_LocationNormalized.astype({'LocationNormalized': 'string'})
df_LocationNormalized.info()
df_LocationNormalized.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   LocationNormalized  244768 non-null  string
dtypes: string(1)
memory usage: 1.9 MB


,LocationNormalized
0,Dorking
1,Glasgow
2,Hampshire
3,Surrey
4,Surrey


### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [44]:
# Data missing sum.
missing = df_LocationNormalized.isnull().sum()
missing

LocationNormalized    0
dtype: int64

In [45]:
# Data missing in percent.
percentMissing = (missing / len(df_LocationNormalized.index)) * 100
percentMissing

LocationNormalized    0.0
dtype: float64

### Aplicando Lower Casing:

In [46]:
df_LocationNormalized["processed_LocationNormalized"] = df_LocationNormalized["LocationNormalized"].str.lower()
df_LocationNormalized.head()

,LocationNormalized,processed_LocationNormalized
0,Dorking,dorking
1,Glasgow,glasgow
2,Hampshire,hampshire
3,Surrey,surrey
4,Surrey,surrey


**NOTE:**  
Eu poderia tentar categorizar ou vectorizar essa variáveis, mas como eu não tenho interesse em trabalhar com essa coluna (feature), vou apenas ignora ela por agora.

---

## 03.6 - Pré-Processando a coluna (feature) "ContractType"
> Essa coluna representa os tipos de contratos por amostra de vaga de emprego, que são **full_time** ou **part_time**.

### Preparando e colocando o tipo de dado mais adequado na coluna (feature) "ContractType":

In [47]:
df_ContractType = full_df[["ContractType"]]
df_ContractType = df_ContractType.astype({'ContractType': 'string'})
df_ContractType.info()
df_ContractType.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ContractType  65442 non-null  string
dtypes: string(1)
memory usage: 1.9 MB


,ContractType
0,<NA>
1,<NA>
2,<NA>
3,<NA>
4,<NA>


### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [48]:
# Data missing sum.
missing = df_ContractType.isnull().sum()
missing

ContractType    179326
dtype: int64

In [49]:
# Data missing in percent.
percentMissing = (missing / len(df_ContractType.index)) * 100
percentMissing

ContractType    73.263662
dtype: float64

**NOTE:**  
Como essa *coluna (feature)* tem 73% dos dados faltantes (missing) vou remové-la. Isso, porque se nós temos mais de 70% dos dados faltando talvez essa variável no nosso modelo seja quase nula.

---

## 03.7 - Pré-Processando a coluna (feature) "ContractTime"
> Tipo de contrato, que pode ser **permanente** ou **contrato**.

### Preparando e colocando o tipo de dado mais adequado na *coluna (feature)* "ContractTime":

In [50]:
df_ContractTime = full_df[["ContractTime"]]
df_ContractTime = df_ContractTime.astype({'ContractTime': 'string'})
df_ContractTime.info()
df_ContractTime.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ContractTime  180863 non-null  string
dtypes: string(1)
memory usage: 1.9 MB


,ContractTime
0,permanent
1,permanent
2,permanent
3,permanent
4,permanent


### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [51]:
# Data missing sum.
missing = df_ContractTime.isnull().sum()
missing

ContractTime    63905
dtype: int64

In [52]:
# Data missing in percent.
percentMissing = (missing / len(df_title.index)) * 100
percentMissing

ContractTime    26.108397
dtype: float64

**NOTE:**  
Para esse caso em específico eu vou remover os valores nulos e categorizar os tipos de contratos:

In [53]:
df_ContractTime = df_ContractTime.dropna()

In [54]:
df_ContractTime.info()
df_ContractTime.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180863 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ContractTime  180863 non-null  string
dtypes: string(1)
memory usage: 2.8 MB


,ContractTime
0,permanent
1,permanent
2,permanent
3,permanent
4,permanent


In [55]:
missing = df_ContractTime.isnull().sum()
missing

ContractTime    0
dtype: int64

In [56]:
# Data missing in percent.
percentMissing = (missing / len(df_ContractTime.index)) * 100
percentMissing

ContractTime    0.0
dtype: float64

In [57]:
df_ContractTime['ContractTime'] = df_ContractTime['ContractTime'].astype('category').cat.codes

In [58]:
df_ContractTime.info()
df_ContractTime.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180863 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   ContractTime  180863 non-null  int8 
dtypes: int8(1)
memory usage: 1.6 MB


,ContractTime
0,1
1,1
2,1
3,1
4,1


In [59]:
df_ContractTime.ContractTime.value_counts()

1    151521
0     29342
Name: ContractTime, dtype: int64

---

## 03.8 - Pré-Processando a coluna (feature) "Company"
> O nome do empregador conforme fornecido pelo anunciante do emprego.

### Preparando e colocando o tipo de dado mais adequado na *coluna (feature)* "Company":

In [60]:
df_Company = full_df[["Company"]]
df_Company = df_Company.astype({'Company': 'string'})
df_Company.info()
df_Company.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Company  212338 non-null  string
dtypes: string(1)
memory usage: 1.9 MB


,Company
0,Gregory Martin International
1,Gregory Martin International
2,Gregory Martin International
3,Gregory Martin International
4,Gregory Martin International
5,Gregory Martin International
6,Indigo 21 Ltd
7,Code Blue Recruitment
8,Code Blue Recruitment
9,Indigo 21 Ltd


### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [61]:
# Data missing sum.
missing = df_Company.isnull().sum()
missing

Company    32430
dtype: int64

In [62]:
# Data missing in percent.
percentMissing = (missing / len(df_title.index)) * 100
percentMissing

Company    13.249281
dtype: float64

**NOTE:**  
Essa coluna (feature) **company** tem os mesmos aspectos da **"ContractTime"**. Mas na minha opinião não vale muito a trabalhar com ela para colocar como uma das entradas do modelo.

---

## 03.9 - Pré-Processando a coluna (feature) "Category"
> Em qual das 30 categorias de empregos padrão este anúncio se encaixa, inferida de uma forma muito confusa com base na origem de onde o anúncio veio.

**NOTE:**  
Sabemos que há muito ruído e erro neste campo.

### Preparando e colocando o tipo de dado mais adequado na *coluna (feature)* "Category":

In [63]:
df_Category = full_df[["Category"]]
df_Category = df_Category.astype({'Category': 'string'})
df_Category.info()
df_Category.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Category  244768 non-null  string
dtypes: string(1)
memory usage: 1.9 MB


,Category
0,Engineering Jobs
1,Engineering Jobs
2,Engineering Jobs
3,Engineering Jobs
4,Engineering Jobs


### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [64]:
# Data missing sum.
missing = df_Category.isnull().sum()
missing

Category    0
dtype: int64

In [65]:
# Data missing in percent.
percentMissing = (missing / len(df_Category.index)) * 100
percentMissing

Category    0.0
dtype: float64

---

## 03.10 - Pré-Processando a coluna (feature) "SalaryRaw"
Imagine que essa coluna representa o salário do anúncio (amostra). Porém:
 - Sem formatação;
 - Com bonus;
 - Remuneração:
   - Por hora;
   - Por mês;
   - Por ano.

**NOTE:**  
Como nós já temos uma variável com o mesmo significado desta, porém, normalizada pelo a **Adzuna** eu vou optar por não trabalhar com essa.

---

## 03.11 - Pré-Processando a coluna (feature) "SalaryNormalized"
Tem o mesmo significado da coluna **"SalaryRaw"**, porém a *Adzuna* normalizou os dados para ser representado de forma **anualizado**.

### Preparando e colocando o tipo de dado mais adequado na *coluna (feature)* "SalaryNormalized":

In [66]:
df_SalaryNormalized = full_df[["SalaryNormalized"]]
df_SalaryNormalized = df_SalaryNormalized.astype({'SalaryNormalized': 'float64'})
df_SalaryNormalized.info()
df_SalaryNormalized.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   SalaryNormalized  244768 non-null  float64
dtypes: float64(1)
memory usage: 1.9 MB


,SalaryNormalized
0,25000.0
1,30000.0
2,30000.0
3,27500.0
4,25000.0


### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [67]:
# Data missing sum.
missing = df_SalaryNormalized.isnull().sum()
missing

SalaryNormalized    0
dtype: int64

In [68]:
# Data missing in percent.
percentMissing = (missing / len(df_SalaryNormalized.index)) * 100
percentMissing

SalaryNormalized    0.0
dtype: float64

---

## 03.12 - Pré-Processando a coluna (feature) "SourceName"
> O nome do site ou anunciante de quem recebemos o anúncio de emprego.

### Preparando e colocando o tipo de dado mais adequado na *coluna (feature)* "SourceName":

In [69]:
df_SourceName = full_df[["SourceName"]]
df_SourceName = df_SourceName.astype({'SourceName': 'string'})
df_SourceName.info()
df_SourceName.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   SourceName  244767 non-null  string
dtypes: string(1)
memory usage: 1.9 MB


,SourceName
0,cv-library.co.uk
1,cv-library.co.uk
2,cv-library.co.uk
3,cv-library.co.uk
4,cv-library.co.uk


### Verificando quanto porcento (%) dos dados são ausentes (missing):

In [70]:
# Data missing sum.
missing = df_SourceName.isnull().sum()
missing

SourceName    1
dtype: int64

In [71]:
# Data missing in percent.
percentMissing = (missing / len(df_SourceName.index)) * 100
percentMissing

SourceName    0.000409
dtype: float64

---

# PARTE 2 - Machine Learning
Agora que nós já aplicamos um **Pré-Processamento** (mesmo que básico) nas colunas (features), vamos iniciar a étapa de modelagem. Ou seja, vamos iniciar o processo de Machine Learning.

> Para a minha modelagem vou focar na coluna (feature) **"FullDescription"**.

Como a nossa métrica avaliação é o [Erro Médio Absoluto](https://en.wikipedia.org/wiki/Mean_absolute_error) vamos focar em algoritmos de regressão.

**Inicialmente, a primeira coisa que vamos fazer é pegar o conjunto de dados de teste:**

In [72]:
with py7zr.SevenZipFile("../datasets/Test_rev1.7z", mode='r') as archive:
  archive.extractall(path="/tmp") # For Linux users.

In [73]:
testing_df = pd.read_csv("/tmp/Test_rev1.csv")

In [74]:
testing_df.info()
testing_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122463 entries, 0 to 122462
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  122463 non-null  int64 
 1   Title               122463 non-null  object
 2   FullDescription     122463 non-null  object
 3   LocationRaw         122463 non-null  object
 4   LocationNormalized  122463 non-null  object
 5   ContractType        33013 non-null   object
 6   ContractTime        90702 non-null   object
 7   Company             106202 non-null  object
 8   Category            122463 non-null  object
 9   SourceName          122463 non-null  object
dtypes: int64(1), object(9)
memory usage: 9.3+ MB


,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SourceName
0,11888454,Business Development Manager,"The Company: Our client is a national training provider based in Gateshead, delivering learning programmes across many regions of England. Founded in **** they have developed a firm foundation that underpins their core offer to employers and individuals that is we work with you to fully understand your training and development needs . Their expertise enables them to deliver a range of learning programmes from NVQ certificates and diplomas to short courses that are designed to upskill individuals, including English and maths. They contract with the Skills Funding Agency to provide Workplace and Classroom based learning programmes, Apprenticeships and courses for individuals who are currently seeking employment or alternative employment. The Role: Our client is looking for an exceptional business development person who could hit the ground running and have a possible client base to bring with them. Selling to businesses local and nationally NVQ and apprenticeship opportunities in Business Admin, Customer Service, Health and Social Care, Plant and Construction, as well as commercial. The Individual: Extremely motivated. Excellent organisation skills. Excellent knowledge of the delivering NVQ s and Apprenticeships. Additional: Mobile phone Laptop 20 days holiday and **** public holidays ****p per mile allowance **** per month car allowance Excellent bonus structure","Tyne Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Teaching Jobs,cv-library.co.uk
1,11988350,Internal Account Manager,"The Company: Founded in **** our client is a UK market leader in the provision of ondemand collaboration software. The solution is a means of improving the project management process, providing a comprehensive toolset for online collaboration, document control, information management and much more. Our client now operates throughout the world, in many diverse markets, with a client list that is the envy of all in the industry. The Role: Our client is seeking to recruit an Internal Account Manager to make a significant contribution to the continued growth and success of the company through the development of new business relationships in the UK construction sector. Joining the established Internal Sales Team you will be responsible for monitoring construction market trends, setting up appointments for the sales team and manage the diary accordingly. You will be required to contact current and potential clients by telephone and book appointments for the Key Account Managers to attend. You will have relationships with all members of the Account Management, Marketing, Development and Technical Support teams; plus current and potential clients. Main Duties: Contact existing and potential clients, contractors and consultants by telephone on a daily basis. Work with Key Account Management team to target all Glennigan other project specific leads and strategically develop new business accounts Manage sales development campaigns Conduct suitable research of markets, clients, competing products and competitor organisations. Build up a database of suitable leads and potential new business contacts. Establish new business relationships through contact via telephone, email and direct mail to both cold and warm leads. Ensure that targets for establishing initial contact with sales leads are met. Set up appointments for the sales team and manage the diary accordingly. Liaise with the field sales team to provide information on leads generated. Respond to customer service telephone calls and emails as required. Resolve, or provide advice on how to resolve, customer queries. Request and respond to relevant information from the client to generate new business opportunities. Send out product information to clients. Assist with the development of robust and accurate proposal and contract documentat

**Agora vamos importar os módulos de regressão para treinar nosso modelo:**

In [82]:
# Regression models
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

# MAE metric.
from sklearn.metrics import mean_absolute_error

**Por fim, vamos treinar e verificar qual dos modelos vai nos dar a melhor métrica - Erro Médio Absoluto (MAE)**

**Regressão Linear:**

In [84]:
lrModel = LinearRegression() # Instance.

In [85]:
# Training.
lrModel.fit(df_FullDescription_vectorized, full_df['SalaryNormalized'])

LinearRegression()

In [92]:
# Saving model.
import joblib
joblib.dump(lrModel, 'lr_model.pkl')
# lrModel = joblib.load('lr_model.pkl') # to load.

['lr_model.pkl']

In [ ]:
# Predictions | testing_df['FullDescription'] = x_test
salary_predicted_ln = lrModel.predict(testing_df)

In [ ]:
# Metric | testing_df['SalaryNormalized'] = y_test, salary_predicted = y_predicted
mae_ln = mean_absolute_error(testing_df['SalaryNormalized'], salary_predicted)
mae_ln

**Ridge Regression:**

In [ ]:
ridgeModel = Ridge(alpha=1.0) # Instance.

In [ ]:
# Training.
ridgeModel.fit(df_FullDescription_vectorized, full_df['SalaryNormalized'])

In [ ]:
# Save the model.
from pycaret.regression import *
save_model(ridgeModel, 'ridge_model')

In [ ]:
# Predictions | testing_df['FullDescription'] = x_test
salary_predicted_ridge = ridgeModel.predict(testing_df['FullDescription'])

In [ ]:
# Metric | testing_df['SalaryNormalized'] = y_test, salary_predicted_ridge = y_predicted
mae_ridge = mean_absolute_error(testing_df['SalaryNormalized'], salary_predicted_ridge)
mae_ridge

**Lasso Regression:**

In [ ]:
lassoModel = Lasso(alpha=10, max_iter=1000, tol=0.1) # Instance.

In [ ]:
# Training.
lassoModel.fit(df_FullDescription_vectorized, full_df['SalaryNormalized'])

In [ ]:
# Save the model.
from pycaret.regression import *
save_model(lassoModel, 'lasso_model')

In [ ]:
# Predictions | testing_df['FullDescription'] = x_test
salary_predicted_lasso = lassoModel.predict(testing_df['FullDescription'])

In [ ]:
# Metric | testing_df['SalaryNormalized'] = y_test, salary_predicted_lasso = y_predicted
mae_lasso = mean_absolute_error(testing_df['SalaryNormalized'], salary_predicted_lasso)
mae_lasso

**Random Forest Regressor:**

In [ ]:
randomforestregressoModel = RandomForestRegressor(n_jobs=-1) # Instance.

In [ ]:
# Training
randomforestregressoModel.fit(df_FullDescription_vectorized, full_df['SalaryNormalized'])

In [ ]:
# Save the model.
from pycaret.regression import *
save_model(randomforestregressoModel, 'randomforestregressoModel_model')

In [ ]:
# Predictions | testing_df['FullDescription'] = x_test
salary_predicted_randomforestregressoModel = randomforestregressoModel.predict(testing_df['FullDescription'])

In [ ]:
# Metric | testing_df['SalaryNormalized'] = y_test, salary_predicted_randomforestregressoModel = y_predicted
mae_lasso = mean_absolute_error(testing_df['SalaryNormalized'], salary_predicted_randomforestregressoModel)
mae_lasso